In [24]:
from pydantic import BaseModel, ValidationError
from datetime import date

class my_class(BaseModel):
    id: int
    name: str
    date_to: date
    date_from: date
    
    
json_row = """
{
    "id": "132",
    "name": "stas",
    "date_to": "2020-01-01",
    "date_from": "2020-01-02"
}
"""
try:
    a = my_class.parse_raw(json_row)
except ValidationError as e:
    print(e)
# print(a)
# print(type(a))
# print(dir(a))


In [14]:
import json
from datetime import date

a = date(2020,1,1)
print(a)
json.dumps(str(a))

2020-01-01


'"2020-01-01"'

In [20]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str|None = "Petia"
    surname: str | None = "Sergevich"
    
user = User(id=123,name="a")
assert user.id == 123
assert isinstance(user.id, int)

print(user.model_dump())
user.id = 321
print(dict(user))

print(user.model_dump_json(indent=4))

{'id': 123, 'name': 'a', 'surname': 'Sergevich'}
{'id': 321, 'name': 'a', 'surname': 'Sergevich'}
{
    "id": 321,
    "name": "a",
    "surname": "Sergevich"
}


In [31]:
from pydantic import BaseModel

class Foo(BaseModel):
    count: int
    size: float|None = None
    
class Bar(BaseModel):
    apple: str = "x"
    banana: str = "y"
    
class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]
    
m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
display(m)
display(m.model_dump())

Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])

{'foo': {'count': 4, 'size': None},
 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}

In [35]:
from pydantic import BaseModel, PydanticUserError


class Foo(BaseModel):
    x: 'Barsic'


try:
    display(Foo.model_json_schema())
except PydanticUserError as e:
    print(e)
    
class Barsic(BaseModel):
    pass

Foo.model_rebuild()
display(Foo.model_json_schema())

`Foo` is not fully defined; you should define `Barsic`, then call `Foo.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.6/u/class-not-fully-defined


{'$defs': {'Barsic': {'properties': {}, 'title': 'Barsic', 'type': 'object'}},
 'properties': {'x': {'$ref': '#/$defs/Barsic'}},
 'required': ['x'],
 'title': 'Foo',
 'type': 'object'}

In [13]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import declarative_base
from typing_extensions import Annotated

from pydantic import BaseModel, ConfigDict, StringConstraints

Base = declarative_base()

class CompanyOrm(Base):
    __tablename__ = 'companies'
    
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(63)))
    

class CompanyModel(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    
    id: int
    public_key: Annotated[str, StringConstraints(max_length=20)]
    name: Annotated[str, StringConstraints(max_length=63)]
    domains: list[Annotated[str, StringConstraints(max_length=255)]]
    
    

co_orm = CompanyOrm(
    id=123,
    public_key="foobaroefhoehoshfoso",
    name="Testing",
    domains=['example.com', 'foobar.com'],
)
print(co_orm)

co_model = CompanyModel.model_validate(co_orm)
print(co_model)
print(type(co_model.public_key))
print(type(co_model.name))

id=123 public_key='foobaroefhoehoshfoso' name='Testing' domains=['example.com', 'foobar.com']
<class 'str'>
<class 'str'>


In [ ]:
import typing

import sqlalchemy as sa
from sqlalchemy.orm import declarative_base

from pydantic import BaseModel, ConfigDict, Field


class MyModel(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    metadata: typing.Dict[str, str] = Field(alias='metadata_')

In [46]:
from pydantic import BaseModel, ConfigDict, Field, AliasGenerator


def to_camel(string: str) -> str:
    return ''.join(word.capitalize() for word in string.split('_'))


class Voice(BaseModel):
    model_config = ConfigDict(
        alias_generator=AliasGenerator(
            validation_alias=lambda field_name: field_name.upper(),
            serialization_alias=lambda field_name: field_name.title(),
        )
    )

    name: str
    language_code: str 

voice = Voice(NAME='Filiz', LANGUAGE_CODE ='tr-TR')
print(voice.language_code)
print(voice)
#> tr-TR
print(voice.model_dump(by_alias=True))
#> {'Name': 'Filiz', 'lang': 'tr-TR'}

tr-TR
name='Filiz' language_code='tr-TR'
{'Name': 'Filiz', 'Language_Code': 'tr-TR'}


In [31]:
from pydantic import BaseModel, Field, AliasPath


class User(BaseModel):
    first_name: str = Field(validation_alias=AliasPath('names', 0))
    last_name: str = Field(validation_alias=AliasPath('names', 1))

user = User.model_validate({'names': ['John', 'Doe']})  
print(user)
#> first_name='John' last_name='Doe'

first_name='John' last_name='Doe'


In [ ]:
from pydantic import BaseModel, Field, AliasChoices


class User(BaseModel):
    first_name: str = Field(validation_alias=AliasChoices('first_name', 'fname'))
    last_name: str = Field(validation_alias=AliasChoices('last_name', 'lname'))

user = User.model_validate({'fname': 'John', 'lname': 'Doe'})  
print(user)
#> first_name='John' last_name='Doe'
user = User.model_validate({'first_name': 'John', 'lname': 'Doe'})  
print(user)
#> first_name='John' last_name='Doe'

In [33]:
from pydantic import BaseModel, Field, AliasPath, AliasChoices


class User(BaseModel):
    first_name: str = Field(validation_alias=AliasChoices('first_name', AliasPath('names', 0)))
    last_name: str = Field(validation_alias=AliasChoices('last_name', AliasPath('names', 1)))


user = User.model_validate({'first_name': 'John', 'last_name': 'Doe'})
print(user)
#> first_name='John' last_name='Doe'
user = User.model_validate({'names': ['John', 'Doe']})
print(user)
#> first_name='John' last_name='Doe'
user = User.model_validate({'names': ['John'], 'last_name': 'Doe'})
print(user)
#> first_name='John' last_name='Doe'

first_name='John' last_name='Doe'
first_name='John' last_name='Doe'
first_name='John' last_name='Doe'


In [36]:
from pydantic import BaseModel, ConfigDict


class Tree(BaseModel):
    model_config = ConfigDict(
        alias_generator=lambda field_name: field_name.upper()
    )

    age: int
    height: float
    kind: str


t = Tree.model_validate({'AGE': 12, 'HEIGHT': 1.2, 'KIND': 'oak'})
print(t.model_dump(by_alias=True))
print(t.model_dump())
#> {'AGE': 12, 'HEIGHT': 1.2, 'KIND': 'oak'}

{'AGE': 12, 'HEIGHT': 1.2, 'KIND': 'oak'}
{'age': 12, 'height': 1.2, 'kind': 'oak'}


In [40]:
from pydantic import AliasGenerator, BaseModel, ConfigDict


class Tree(BaseModel):
    model_config = ConfigDict(
        alias_generator=AliasGenerator(
            validation_alias=lambda field_name: field_name.upper(),
            serialization_alias=lambda field_name: field_name.title(),
        )
    )

    age: int
    height: float
    kind: str


t = Tree.model_validate({'AGE': 12, 'HEIGHT': 1.2, 'KIND': 'oak'})
print(t.model_dump(by_alias=True))
print(t.model_dump())
#> {'Age': 12, 'Height': 1.2, 'Kind': 'oak'}

{'Age': 12, 'Height': 1.2, 'Kind': 'oak'}
{'age': 12, 'height': 1.2, 'kind': 'oak'}


In [ ]:
from typing import List

from pydantic import BaseModel, ConfigDict


class PetCls:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species


class PersonCls:
    def __init__(self, *, name: str, age: float = None, pets: List[PetCls]):
        self.name = name
        self.age = age
        self.pets = pets


class Pet(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    name: str
    species: str


class Person(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    name: str
    age: float = None
    pets: List[Pet]


bones = PetCls(name='Bones', species='dog')
orion = PetCls(name='Orion', species='cat')
anna = PersonCls(name='Anna', age=20, pets=[bones, orion])
anna_model = Person.model_validate(anna)
print(anna_model)
"""
name='Anna' age=20.0 pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')]
"""

In [8]:
from typing import List

from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    list_of_ints: List[int]
    a_float: float


data = dict(
    list_of_ints=['1', 2, 3],
    a_float='1.2',
)

try:
    display(Model(**data).model_dump_json())
except ValidationError as e:
    print(e)

'{"list_of_ints":[1,2,3],"a_float":1.2}'

In [10]:
from pydantic import BaseModel


class Model(BaseModel):
    a: int


m = Model(a=0)
display(m)
# note: the `model_config` setting validate_assignment=True` can prevent this kind of misbehavior
m.a = 'not an int'
display(m)
# doesn't raise a validation error even though m is invalid
m2 = Model.model_validate(m)
display(m2)

Model(a=0)

Model(a='not an int')

Model(a='not an int')

In [11]:
from pydantic import BaseModel, ConfigDict, ValidationError


class Model(BaseModel):
    a: int

    model_config = ConfigDict(revalidate_instances='always')


m = Model(a=0)
display(m)
# note: the `model_config` setting validate_assignment=True` can prevent this kind of misbehavior
m.a = 'not an int'
display(m)

try:
    m2 = Model.model_validate(m)
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    a
      Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not an int', input_type=str]
    """

Model(a=0)

Model(a='not an int')

1 validation error for Model
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not an int', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/int_parsing


In [19]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    age: int
    name: str = 'John Doe'


original_user = User(id=123, age=32)

user_data = original_user.model_dump()
print(user_data)
#> {'id': 123, 'age': 32, 'name': 'John Doe'}
fields_set = original_user.model_fields_set
print(fields_set)
#> {'age', 'id'}

# ...
# pass user_data and fields_set to RPC or save to the database etc.
# ...

# you can then create a new instance of User without
# re-running validation which would be unnecessary at this point:
new_user = User.model_construct(_fields_set=fields_set, **user_data)
new_user2 = User.model_construct( **user_data)
print(repr(new_user))
print(repr(new_user2))
#> User(id=123, age=32, name='John Doe')
print(new_user.model_fields_set)
print(new_user2.model_fields_set)
#> {'age', 'id'}

# construct can be dangerous, only use it with validated data!:
bad_user = User.model_construct(id='dog')
print(repr(bad_user))
#> User(id='dog', name='John Doe')

{'id': 123, 'age': 32, 'name': 'John Doe'}
{'age', 'id'}
User(id=123, age=32, name='John Doe')
User(id=123, age=32, name='John Doe')
{'age', 'id'}
{'age', 'name', 'id'}
User(id='dog', name='John Doe')


In [2]:
from typing import Any, Generic, Tuple, Type, TypeVar

from pydantic import BaseModel

DataT = TypeVar('DataT', bound=str)


class Response(BaseModel, Generic[DataT]):
    data: DataT

    @classmethod
    def model_parametrized_name(cls, params: Tuple[Type[Any], ...]) -> str:
        return f'{params[0].__name__.title()}Response'


print(repr(Response[int](data=1)))
#> IntResponse(data=1)
print(repr(Response[str](data='a')))
#> StrResponse(data='a')

IntResponse(data=1)
StrResponse(data='a')


In [6]:
from typing import Generic, List, Optional, TypeVar

from pydantic import BaseModel, ValidationError

DataT = TypeVar('DataT')


class DataModel(BaseModel):
    numbers: List[int]
    people: List[str]


class Response(BaseModel, Generic[DataT]):
    data: Optional[DataT] = None


print(Response[int](data=1))
#> data=1
print(Response[str](data='value'))
#> data='value'
print(Response[str](data='value').model_dump())
#> {'data': 'value'}

data = DataModel(numbers=[1, 2, 3], people=[])
print(Response[DataModel](data=data).model_dump())
#> {'data': {'numbers': [1, 2, 3], 'people': []}}
try:
    print(Response[int](data='13'))
except ValidationError as e:
    print(e)

data=1
data='value'
{'data': 'value'}
{'data': {'numbers': [1, 2, 3], 'people': []}}
data=13


In [7]:
from typing import Generic

from typing_extensions import TypeVar

from pydantic import BaseModel

TBound = TypeVar('TBound', bound=BaseModel)
TNoBound = TypeVar('TNoBound')


class IntValue(BaseModel):
    value: int


class ItemBound(BaseModel, Generic[TBound]):
    item: TBound


class ItemNoBound(BaseModel, Generic[TNoBound]):
    item: TNoBound


item_bound_inferred =    ItemBound(item=IntValue(value=3))
item_bound_explicit =    ItemBound[IntValue](item=IntValue(value=3))
item_no_bound_inferred = ItemNoBound(item=IntValue(value=3))
item_no_bound_explicit = ItemNoBound[IntValue](item=IntValue(value=3))

display(item_bound_inferred)
display(item_bound_explicit)
display(item_no_bound_inferred)
display(item_no_bound_explicit)

ItemBound(item=IntValue(value=3))

ItemBound[IntValue](item=IntValue(value=3))

ItemNoBound(item=IntValue(value=3))

ItemNoBound[IntValue](item=IntValue(value=3))

In [12]:
from typing import Generic, Optional

from typing_extensions import TypeVar

from pydantic import BaseModel, SerializeAsAny


class ErrorDetails(BaseModel):
    foo: str


ErrorDataT = TypeVar('ErrorDataT', default=ErrorDetails)


class Error(BaseModel, Generic[ErrorDataT]):
    message: str
    details: Optional[ErrorDataT]


class MyErrorDetails(ErrorDetails):
    bar: str


# serialized using the default's serializer
error = Error(
    message='We just had an error',
    details=MyErrorDetails(foo='var', bar='var2'),
)
display(error.model_dump())
assert error.model_dump() == {
    'message': 'We just had an error',
    'details': {
        'foo': 'var',
    },
}


class SerializeAsAnyError(BaseModel, Generic[ErrorDataT]):
    message: str
    details: Optional[SerializeAsAny[ErrorDataT]]


# serialized as Any
error = SerializeAsAnyError(
    message='We just had an error',
    details=MyErrorDetails(foo='var', bar='baz'),
)
display(error.model_dump())
assert error.model_dump() == {
    'message': 'We just had an error',
    'details': {
        'foo': 'var',
        'bar': 'baz',
    },
}

{'message': 'We just had an error', 'details': {'foo': 'var'}}

{'message': 'We just had an error', 'details': {'foo': 'var', 'bar': 'baz'}}

In [ ]:
from pydantic import BaseModel, ConfigDict, Field


class User(BaseModel):
    model_config = ConfigDict(populate_by_name=True)

    name: str = Field(alias='full_name')  
    age: int


user = User(full_name='John Doe', age=20)  
print(user)
#> name='John Doe' age=20
user = User(name='John Doe', age=20)  
print(user)
#> name='John Doe' age=20